# Duke Nutrition Assistant CS 372 Project - Notebook 3: LLM Integration
## Step 3: Connect Retrieval to GPT-4 for Complete RAG System

**Goal of this notebook:**
- Loads embeddings from Notebook 2
- Connects to OpenAI GPT-4 API
- Builds complete RAG system (retrieve + generate)
- Tests with real nutrition queries

**Rubric items targetted:**
- Made API calls to state-of-the-art model (5 pts)
- Built RAG system with retrieval + generation (10 pts)
- Multi-turn conversation with context (7 pts)
- Applied prompt engineering (3 pts)
- In-context learning with few-shot examples (5 pts)

## Setup & Installation

In [536]:
!pip install openai --break-system-packages --quiet

print(" OpenAI library installed")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 OpenAI library installed


In [538]:
import json
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from openai import OpenAI
import os
from datetime import datetime

print("Imports successful!")

Imports successful!


## Set Up API Key

In [545]:
OPENAI_API_KEY = ""

client = OpenAI(api_key=OPENAI_API_KEY)

try:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": "Say 'API key works!'"}],
        max_tokens=10
    )
    print(" API Key is valid")
    print(f"Response: {response.choices[0].message.content}")
except Exception as e:
    print(" API Key error")
    print(f"Error: {e}")

 API Key is valid
Response: API key works!


## Load Your Data & Embeddings

In [550]:
with open('data/menu_processed.json', 'r') as f:
    data = json.load(f)

documents = data['documents']
items = data['items']

print(f"{len(documents):,} menu items")

1,644 menu items


In [554]:
#Load embeddings from Notebook 2
menu_embeddings = np.load('models/menu_embeddings.npy')

print(f" Loaded embeddings: {menu_embeddings.shape}")
print(f"   Shape: ({len(documents)} items, 384 dimensions)")

 Loaded embeddings: (1637, 384)
   Shape: (1644 items, 384 dimensions)


In [560]:
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"

if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

embedding_tokenizer = AutoTokenizer.from_pretrained(embedding_model_name)
embedding_model = AutoModel.from_pretrained(embedding_model_name).to(device)
embedding_model.eval()

print(f"Embedding model loaded on {device} -Success")

Embedding model loaded on mps -Success


## Retrieval Function (from Notebook 2)

In [563]:
def compute_embeddings(texts, model, tokenizer, batch_size=32, device="mps"):

    embeddings = []
    
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            
            inputs = tokenizer(batch, 
                             return_tensors='pt',
                             padding=True, 
                             truncation=True,
                             max_length=512)
            
            inputs = {k: v.to(device) for k, v in inputs.items()}
            outputs = model(**inputs)
            pooled = outputs.last_hidden_state.mean(dim=1)
            embeddings.append(pooled.cpu().numpy())
    
    return np.vstack(embeddings)


def retrieve_top_k(query, context_embeddings, contexts, model, tokenizer, device, k=5):
    """
    Retrieve top-k most similar menu items.
    FROM RAG HOMEWORK
    """
    query_embedding = compute_embeddings([query], model, tokenizer, 
                                        batch_size=1, device=device)[0]
    
    #Calculate cosine similarities
    dot_product = np.dot(context_embeddings, query_embedding)
    norms = (np.linalg.norm(context_embeddings, axis=1) * 
            np.linalg.norm(query_embedding))
    similarities = dot_product / norms
    
    #Get top-k indices
    top_indices = np.argsort(similarities)[-k:][::-1]
    
    return [
        {
            'index': int(idx),
            'document': contexts[idx],
            'item_name': items[idx]['item_name'],
            'restaurant': items[idx]['restaurant'],
            'score': float(similarities[idx]),
            'item': items[idx]  # Full item details
        }
        for idx in top_indices
    ]

print(" Retrieval functions ready!")

 Retrieval functions ready!


## RAG System Class

This combines retrieval + generation into one system

## Initialize RAG System

In [647]:
import re
import numpy as np
import torch

class DukeNutritionRAG:
    """
    Complete RAG system for Duke nutrition recommendations.
    """
    
    def __init__(self, client, embeddings, documents, items, 
                 embedding_model, embedding_tokenizer, device):
        self.client = client
        self.embeddings = embeddings
        self.documents = documents
        self.items = items
        self.embedding_model = embedding_model
        self.embedding_tokenizer = embedding_tokenizer
        self.device = device
        self.conversation_history = []
        self.dietary_requirement = None  
        self.excluded_restaurants = []  
        self.included_restaurants = []
        self.nutrition_goal = None    
        
        self.system_prompt = """You are a helpful nutrition assistant for Duke University students.

Your job is to recommend ACTUAL MEALS from Duke dining halls based on students' nutrition goals.

CRITICAL: When recommending meals, PRIORITIZE MACRO RATIOS, not just absolute values:
- For cutting/weight loss: Prioritize items with HIGH protein-to-calorie ratio (≥40%) AND LOW TOTAL CALORIES (<400 cal ideal)
  Example: Grilled Chicken (45g protein, 250 cal, 72% protein) is BETTER for cutting than
  Chicken Wings (68g protein, 720 cal, 38% protein) - the wings have too many calories even with good protein!
  
- For clean bulking: Prioritize items with MODERATE protein ratio (30-40%) and adequate calories
  Example: Salmon bowl (35g protein, 450 cal, 31% protein) is ideal for lean gains

- For post-workout recovery: Prioritize items with HIGH ABSOLUTE PROTEIN (30-50g total grams!) for muscle recovery
  Example: Grilled Chicken (45g protein) or Steak (40g protein) - need high protein to rebuild muscle after training!

- For meals with more fiber: Prioritize items with the HIGHEST fiber content available
  Example: Oatmeal with 10g fiber is excellent for digestive health and satiety
  
- For keto: Prioritize items with HIGH fat ratio (≥60%) and LOW carb ratio (<10%)
  Example: Avocado bowl (70% fat, 5% carbs) beats low-fat options
  
- For endurance: Prioritize items with HIGH carb ratio (≥60%)
  Example: Pasta (67% carbs) beats protein-heavy meals for pre-run

The RATIO matters more than absolute grams when calories are a concern! CONTEXT IS SUPER IMPORTANT, make sure to understand the food item before recommending.

IMPORTANT RULES
- If the query says they can't eat at a restaurant, don't even list the option there at all, and please recommend the next best option from the place that is accepted. 
- If the query says they only want foods from restaurant(s) only recommend foods from there, you can just filter by the next best ones
- Only recommend complete meals (entrees, sandwiches, salads, breakfast items)
- WHen query mentions like "high fiber" or "high protein" etc. please provide the grams or ratio of fiber or protein or whatever in the response. For fiber specifically, ALWAYS mention the exact grams prominently.
- NEVER recommend protein powders, supplements, or condiments
- If you see items like "Whey Protein" or "Powdered Sugar", IGNORE them completely because they are likely not actual food items(this is why its crucial for you to understand the items before recommending)
- Explain WHY each item matches their goal (mention macro ratios when relevant)
- Be specific about which dining hall has each item
- Be conversational and friendly
- Recommend 3 food items per query
- Try to avoid recommending plain words that seem like they might be a base and not an actual meal/food like "Spaghetti" is likely just a base right, while "Spaghetti with Meatballs" is definitely a full meal
- Make sure to actually understand the food you're recommending (gather what it is and understand it) and make sure it makes sense to what the user is requesting.
- Keep responses concise (2-4 sentences per recommendation)

If you see macro percentages in the item details, USE THEM to make better recommendations!

Example responses:
Example 1: 
User: "high pr
"For cutting, I recommend the Grilled Chicken at Farmstead (45g protein, 250 cal, 72% protein). 
It's incredibly lean and perfect for preserving muscle while losing fat. You could also try 
the Turkey Breast at J.B.'s (38g protein, 180 cal, 84% protein) for an even leaner option! For something lighter, you could try..."
Example 2:
User: "keto friendly meal"
Assistant: "For keto, I recommend the Avocado Bowl at Il Forno 
(70% fat, 5% carbs). Perfect macro split for ketosis..."

Example 3:
User: "high fiber breakfast"
Assistant: "For fiber, try the Oatmeal at Marketplace (10g fiber). 
Excellent for digestive health..."

Make sure to be flexible with the accepting types of queries."""
    
    def _compute_embedding(self, text):
        """Compute embedding for a single text."""
        inputs = self.embedding_tokenizer(
            text, 
            return_tensors="pt", 
            truncation=True, 
            max_length=512, 
            padding=True
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.embedding_model(**inputs)
            embedding = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        
        return embedding.flatten()
    
    def _cosine_similarity(self, a, b):
        """Calculate cosine similarity between two vectors."""
        return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
    
    def _is_actual_meal(self, item):
        """Filter out non-meals (condiments, powders, etc.)."""
        name = item.get('item_name', '').lower()
        
        exclude_keywords = [
            'powder', 'powdered', 'sugar', 'syrup', 'honey',
            'salt', 'pepper', 'sauce', 'dressing', 'spread',
            'butter', 'oil', 'vinegar', 'seasoning',
            'whey protein', 'protein powder', 'boost', 'supplement',
            'condiment', 'topping', 'sprinkles',
            'mayo', 'vinaigrette', 'shot', 'espresso',
            'lettuce', 'spinach', 'kale', 'arugula',  #Salad bases, not meals
            'tomato', 'onion', 'pickle', 'cucumber',  #Toppings, not meals
            'cheese slice', 'american cheese', 'cheddar cheese'  #Toppings
        ]
        
        for keyword in exclude_keywords:
            if keyword in name:
                return False
        
        return True
    
    def _identify_excluded_restaurants(self, query):
        """Identify restaurants to exclude based on query."""
        query_lower = query.lower()
        excluded = []
        
        # Complete patterns for ALL 27 Duke dining locations
        restaurant_patterns = [
            # Main dining halls
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?marketplace', 'Marketplace'),
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?(?:the\s+)?farmstead', 'The Farmstead'),
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?trinity', 'Trinity Cafe'),
            
            # Quick service
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?il\s*forno', 'Il Forno'),
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?sprout', 'Sprout'),
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?(?:the\s+)?skillet', 'The Skillet'),
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?tandoor', 'Tandoor Indian Cuisine'),
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?ginger', 'Ginger + Soy'),
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?sazon', 'Sazon'),
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?gyotaku', 'Gyotaku'),
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?thyme', "It's Thyme"),
            
            # Specialty
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?j\.?b\.?\'?s', "J.B.'s Roast & Chops"),
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?gothic', 'Gothic Grill'),
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?pitchfork', 'The Pitchfork'),
            
            # Coffee shops
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?beyu', 'Beyu Blue Coffee'),
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?bseisu', 'Bseisu Coffee Bar'),
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?freeman', 'Freeman Café'),
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?nasher', 'Nasher Museum Café'),
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?zweli', "Zweli's Café at Duke Divinity"),
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?devils?\s+krafthouse', 'The Devils Krafthouse'),
            
            # Delis
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?sanford', 'Sanford Deli'),
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?saladalia', 'Saladalia @ The Perk'),
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?bella', 'Bella Union'),
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?twinnie', "Twinnie's"),
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?red\s+mango', 'Red Mango'),
            
            # Special
            (r'(?:no|not|exclude)\s+(?:meals?\s+(?:at|from)\s+)?marine\s+lab', 'Duke Marine Lab'),
        ]
        
        for pattern, restaurant in restaurant_patterns:
            if re.search(pattern, query_lower):
                excluded.append(restaurant)
        
        return excluded
    
    def _identify_included_restaurants(self, query):
        """Identify restaurants to ONLY show based on query."""
        query_lower = query.lower()
        included = []
        
        # Complete patterns for ALL 27 Duke dining locations
        restaurant_patterns = [
            # Main dining halls
            (r'(?:from|at|only\s+at)\s+marketplace', 'Marketplace'),
            (r'(?:from|at|only\s+at)\s+(?:the\s+)?farmstead', 'The Farmstead'),
            (r'(?:from|at|only\s+at)\s+trinity(?:\s+cafe)?', 'Trinity Cafe'),
            
            # Quick service & cafes
            (r'(?:from|at|only\s+at)\s+il\s*forno', 'Il Forno'),
            (r'(?:from|at|only\s+at)\s+sprout', 'Sprout'),
            (r'(?:from|at|only\s+at)\s+(?:the\s+)?skillet', 'The Skillet'),
            (r'(?:from|at|only\s+at)\s+tandoor', 'Tandoor Indian Cuisine'),
            (r'(?:from|at|only\s+at)\s+ginger(?:\s*\+?\s*soy)?', 'Ginger + Soy'),
            (r'(?:from|at|only\s+at)\s+sazon', 'Sazon'),
            (r'(?:from|at|only\s+at)\s+gyotaku', 'Gyotaku'),
            (r'(?:from|at|only\s+at)\s+(?:it\'?s\s+)?thyme', "It's Thyme"),
            
            # Specialty restaurants
            (r'(?:from|at|only\s+at)\s+j\.?b\.?\'?s', "J.B.'s Roast & Chops"),
            (r'(?:from|at|only\s+at)\s+gothic\s+grill', 'Gothic Grill'),
            (r'(?:from|at|only\s+at)\s+(?:the\s+)?pitchfork', 'The Pitchfork'),
            
            # Coffee shops
            (r'(?:from|at|only\s+at)\s+beyu(?:\s+blue)?(?:\s+coffee)?', 'Beyu Blue Coffee'),
            (r'(?:from|at|only\s+at)\s+bseisu', 'Bseisu Coffee Bar'),
            (r'(?:from|at|only\s+at)\s+freeman(?:\s+caf[eé])?', 'Freeman Café'),
            (r'(?:from|at|only\s+at)\s+nasher(?:\s+museum)?(?:\s+caf[eé])?', 'Nasher Museum Café'),
            (r'(?:from|at|only\s+at)\s+zweli\'?s', "Zweli's Café at Duke Divinity"),
            (r'(?:from|at|only\s+at)\s+(?:the\s+)?devils?\s+krafthouse', 'The Devils Krafthouse'),
            
            # Delis & quick serve
            (r'(?:from|at|only\s+at)\s+sanford\s+deli', 'Sanford Deli'),
            (r'(?:from|at|only\s+at)\s+saladalia', 'Saladalia @ The Perk'),
            (r'(?:from|at|only\s+at)\s+(?:the\s+)?perk', 'Saladalia @ The Perk'),
            (r'(?:from|at|only\s+at)\s+bella\s+union', 'Bella Union'),
            (r'(?:from|at|only\s+at)\s+twinnie\'?s', "Twinnie's"),
            (r'(?:from|at|only\s+at)\s+red\s+mango', 'Red Mango'),
            
            # Special locations
            (r'(?:from|at|only\s+at)\s+duke\s+marine\s+lab', 'Duke Marine Lab'),
            
            # Generic (only if no specific match)
            (r'(?:from|at|only\s+at)\s+(?:the\s+)?cafe(?!\s)', 'Cafe'),  # Match "cafe" but not "cafe something"
        ]
        
        for pattern, restaurant in restaurant_patterns:
            if re.search(pattern, query_lower):
                included.append(restaurant)
        
        return included
    
    def _detect_nutrition_goal(self, query):
        """Detect nutrition goal from query (for ratio bonuses)."""
        query_lower = query.lower()
        
        #Check for post-workout
        if any(word in query_lower for word in ['post-workout', 'post workout', 'after workout', 'recovery meal', 'after gym', 'after training']):
            return 'post-workout'
        #Check for cutting/weight loss
        elif any(word in query_lower for word in ['cutting', 'lean', 'weight loss', 'lose weight', 'lose fat', 'cut']):
            return 'cutting'
        #Check for bulking/muscle gain
        elif any(word in query_lower for word in ['bulk', 'gain', 'muscle building', 'mass']):
            return 'bulking'
        #Check for keto
        elif any(word in query_lower for word in ['keto', 'low carb', 'high fat']):
            return 'keto'
        #Check for high fiber
        elif any(word in query_lower for word in ['fiber', 'high fiber', 'digestive', 'gut health']):
            return 'fiber'
        #Check for endurance
        elif any(word in query_lower for word in ['endurance', 'marathon', 'run', 'energy', 'carb', 'cardio']):
            return 'endurance'
        
        return None
    
    def _detect_dietary_requirement(self, query):
        """Detect dietary requirements from query."""
        query_lower = query.lower()
        
        if 'vegan' in query_lower:
            return 'vegan'
        elif 'vegetarian' in query_lower:
            return 'vegetarian'
        elif 'halal' in query_lower:
            return 'halal'
        elif 'gluten free' in query_lower or 'gluten-free' in query_lower:
            return 'gluten free'
        
        return None
    
    def _matches_dietary_requirement(self, item, requirement):
        """Check if item matches dietary requirement."""
        dietary_labels = str(item.get('dietary_labels', '')).lower()
        
        if requirement == 'vegan':
            return 'vegan' in dietary_labels
        elif requirement == 'vegetarian':
            return 'vegetarian' in dietary_labels or 'vegan' in dietary_labels
        elif requirement == 'halal':
            return 'halal' in dietary_labels
        elif requirement == 'gluten free':
            return 'gluten free' in dietary_labels or 'gluten-free' in dietary_labels
        
        return False
    
    def _calculate_ratio_score(self, item, query, goal=None):
        """Calculate bonus score based on macro ratios for query context."""
        query_lower = query.lower()
        
        # Use saved goal if not provided
        if not goal and self.nutrition_goal:
            goal = self.nutrition_goal
        
        try:
            protein = float(item.get('protein_g', 0))
            carbs = float(item.get('total_carbs_g', 0))
            fat = float(item.get('total_fat_g', 0))
            fiber = float(item.get('fiber_g', 0))
            calories = float(item.get('calories', 1))
            
            if calories == 0:
                return 0
            
            protein_pct = (protein * 4 / calories) * 100
            carbs_pct = (carbs * 4 / calories) * 100
            fat_pct = (fat * 9 / calories) * 100
            
        except (ValueError, TypeError, ZeroDivisionError):
            return 0
        
        # POST-WORKOUT: Prioritize HIGH ABSOLUTE PROTEIN (30-50g) for muscle recovery
        # also want decent carbs for glycogen replenishment
        if goal == 'post-workout' or (not goal and any(word in query_lower for word in ['post-workout', 'post workout', 'after workout', 'recovery'])):
            if protein >= 40:  # Excellent protein for recovery
                return 0.7  # MASSIVE bonus!
            elif protein >= 30:  # Good protein for recovery
                return 0.5
            elif protein >= 20:  # Decent protein
                return 0.3
            elif protein < 15:  # Too low for post-workout
                return -0.3  # PENALTY for low protein!
        
        # Ue goal if provided, otherwise check query
        if goal == 'cutting' or (not goal and any(word in query_lower for word in ['cutting', 'lean', 'weight loss', 'lose weight', 'lose fat'])):
            # For cutting: penalize high-calorie items!
            if protein_pct >= 40 and calories < 400:
                return 0.4  # Perfect cutting food: high protein %, low calories
            elif protein_pct >= 40 and calories < 600:
                return 0.2  # Good protein but moderate calories
            elif protein_pct >= 30 and calories < 400:
                return 0.25
            elif protein_pct >= 30 and calories < 600:
                return 0.1
            elif calories > 600:
                return -0.2  #PENALTY for high-calorie items when cutting!
        
        if goal == 'bulking' or (not goal and any(word in query_lower for word in ['bulk', 'gain', 'muscle building'])):
            if 30 <= protein_pct <= 40 and calories >= 300:
                return 0.25
            elif protein_pct >= 25:
                return 0.1
        
        if goal == 'keto' or (not goal and any(word in query_lower for word in ['keto', 'low carb', 'high fat'])):
            if fat_pct >= 60 and carbs_pct < 10:
                return 0.35
            elif fat_pct >= 50:
                return 0.2
        
        # FIBER: Use absolute grams and not ratio, Fiber has ~0 calories anyway
        #MASSIVE bonuses because fiber should dominate the query
        if goal == 'fiber' or (not goal and any(word in query_lower for word in ['fiber', 'high fiber', 'digestive'])):
            if fiber >= 8:  # Excellent fiber (8+ grams)
                return 0.6  # HUGE bonus, will beat most semantic matches
            elif fiber >= 5:  # Good fiber(5-7 grams)
                return 0.4
            elif fiber >= 3:  # Decent fiber(3-4 grams)
                return 0.2
        
        if goal == 'endurance' or (not goal and any(word in query_lower for word in ['endurance', 'marathon', 'run', 'energy', 'carb'])):
            if carbs_pct >= 60:
                return 0.3
            elif carbs_pct >= 50:
                return 0.15
        
        return 0
    
    def reset_conversation(self):
        """Reset conversation-specific memory."""
        self.conversation_history = []
        self.dietary_requirement = None
        self.excluded_restaurants = []
        self.included_restaurants = []
        self.nutrition_goal = None

    def retrieve(self, query, k=5):
        """Retrieve top k relevant items with filtering and bonuses."""
        #Detect and save dietary requirement
        dietary_req = self._detect_dietary_requirement(query)
        if dietary_req:
            self.dietary_requirement = dietary_req
        elif self.dietary_requirement:
            dietary_req = self.dietary_requirement
        
        #Detect and save nutrition goal
        nutrition_goal = self._detect_nutrition_goal(query)
        if nutrition_goal:
            self.nutrition_goal = nutrition_goal
        
        #Detect and save excluded restaurants
        excluded_now = self._identify_excluded_restaurants(query)
        if excluded_now:
            #Add to existing exclusions(avoid duplicates)
            for restaurant in excluded_now:
                if restaurant not in self.excluded_restaurants:
                    self.excluded_restaurants.append(restaurant)
        
        # Use higher multiplier for dietary/fiber/post-workout to cast wider net
        if dietary_req:
            multiplier = 10
        elif nutrition_goal == 'fiber':
            multiplier = 8  # Cast wider net for fiber queries!
        elif nutrition_goal == 'post-workout':
            multiplier = 8  # Cast wider net for high-protein items!
        else:
            multiplier = 4
        
        #Compute query embedding
        query_embedding = self._compute_embedding(query)
        
        #Calculate similarities
        raw_results = []
        for i, doc_embedding in enumerate(self.embeddings):
            similarity = self._cosine_similarity(query_embedding, doc_embedding)
            raw_results.append({
                'item': self.items[i],
                'score': similarity,
                'base_similarity': similarity,
                'ratio_bonus': 0
            })
        
        #Sort by similarity
        raw_results.sort(key=lambda x: x['score'], reverse=True)
        raw_results = raw_results[:k*multiplier]
        
        #Filter non-meals + dietary
        filtered_meals = []
        for result in raw_results:
            item = result['item']
            
            #Hard dietary filter
            if dietary_req:
                if not self._matches_dietary_requirement(item, dietary_req):
                    continue
            
            if self._is_actual_meal(item):
                filtered_meals.append(result)
        
        #Filter excluded restaurants (use saved list!)
        if self.excluded_restaurants:
            filtered_meals = [
                r for r in filtered_meals 
                if r['item'].get('restaurant') not in self.excluded_restaurants
            ]
        included_now = self._identify_included_restaurants(query)

        if included_now:
            # Save included restaurants
            for restaurant in included_now:
                if restaurant not in self.included_restaurants:
                    self.included_restaurants.append(restaurant)

        # Apply included filter (overrides excluded if both present)
        if self.included_restaurants:
            filtered_meals = [
                r for r in filtered_meals
                if r["item"].get("restaurant") in self.included_restaurants
            ]
        
        #Apply ratio bonuses(using saved goal!)
        for result in filtered_meals:
            bonus = self._calculate_ratio_score(result['item'], query, goal=self.nutrition_goal)
            result['ratio_bonus'] = bonus
            result['score'] = result['base_similarity'] + bonus
        
        #Re-sort by total score
        filtered_meals.sort(key=lambda x: x['score'], reverse=True)
        
        #Deduplication
        seen_names = set()
        unique_results = []
        
        for result in filtered_meals:
            name = result['item']['item_name']
            if name not in seen_names:
                seen_names.add(name)
                unique_results.append(result)
                if len(unique_results) >= k:
                    break
        
        #ERROR HANDLING:If no results found, try again with relaxed filters
        if len(unique_results) == 0:
            #Fall back to just similarity without dietary/restaurant filters
            for result in raw_results[:k*2]:
                if self._is_actual_meal(result['item']):
                    unique_results.append(result)
                    if len(unique_results) >= k:
                        break
        
        return unique_results
    
    def format_context(self, retrieved_items):
        """Format retrieved items as context for LLM."""
        context_parts = []
        
        for i, result in enumerate(retrieved_items, 1):
            item = result['item']
            name = item['item_name']
            restaurant = item.get('restaurant', 'Unknown')
            calories = item.get('calories', 'N/A')
            protein = item.get('protein_g', 'N/A')
            carbs = item.get('total_carbs_g', 'N/A')
            fat = item.get('total_fat_g', 'N/A')
            fiber = item.get('fiber_g', 'N/A')
            
            context_parts.append(
                f"{i}. {name} at {restaurant}\n"
                f"   - Calories: {calories}\n"
                f"   - Protein: {protein}g, Carbs: {carbs}g, Fat: {fat}g, Fiber: {fiber}g"
            )
        
        return "\n\n".join(context_parts)
    
    def ask(self, query, k=5, use_history=False, verbose=False):
        """Main method to get recommendations."""
        #Retrieve relevant items
        retrieved_items = self.retrieve(query, k=k)
        
        if verbose:
            print(f"\n🔍 Retrieved {len(retrieved_items)} items:")
            for result in retrieved_items:
                item = result['item']
                print(f"   - {item['item_name']} (score: {result['score']:.3f})")
        
        #Format context
        context = self.format_context(retrieved_items)
        
        #Build messages
        messages = [{"role": "system", "content": self.system_prompt}]
        
        if use_history:
            messages.extend(self.conversation_history)
        
        messages.append({
            "role": "user",
            "content": f"Based on these Duke dining hall items:\n\n{context}\n\nUser query: {query}"
        })
        
        #Get LLM response
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0.7,
            max_tokens=500
        )
        
        answer = response.choices[0].message.content
        
        #Update history if needed
        if use_history:
            self.conversation_history.append({"role": "user", "content": query})
            self.conversation_history.append({"role": "assistant", "content": answer})
        
        return {
            'response': answer,
            'retrieved_items': retrieved_items
        }

In [649]:
#Create RAG system
rag = DukeNutritionRAG(
    client=client,
    embeddings=menu_embeddings,
    documents=documents,
    items=items,
    embedding_model=embedding_model,
    embedding_tokenizer=embedding_tokenizer,
    device=device
)

print("RAG system initialized!")
print("\nReady to answer nutrition queries")

RAG system initialized!

Ready to answer nutrition queries


## Test Single Query

In [652]:
# Test with a simple query
result = rag.ask(
    "I want something high in protein and low in calories like a meal",
    k=5,
    use_history=False,
    verbose=True
)

print("="*80)
print(" RESPONSE:")
print("="*80)
print(result['response'])
print("="*80)


🔍 Retrieved 5 items:
   - Calabash Fish Sandwich (score: 0.655)
   - Chorizo (score: 0.640)
   - Mocha Frappe Whole Milk (score: 0.638)
   - Morty's Reuben on Rye Sandwich (score: 0.635)
   - Tostones (score: 0.635)
 RESPONSE:
For a high-protein, low-calorie meal, I recommend the Chorizo at The Pitchfork (19g protein, 360 cal, 21% protein). It's a good choice with a decent protein-to-calorie ratio and relatively low carbs. 

Another option is the Tostones at Sazon, which, while low in protein (2g), can be paired with a higher protein item for a balanced meal. However, I suggest focusing mainly on the Chorizo for your main protein source. 

Unfortunately, the other options like the Calabash Fish Sandwich and Morty's Reuben are too high in calories for your preference, but sticking with the Chorizo should help meet your goal!


In [653]:
# Test with a simple query
result = rag.ask(
    "I want dinner that has some good protein for digestion",
    k=5,
    use_history=False,
    verbose=True
)

print("="*80)
print(" RESPONSE:")
print("="*80)
print(result['response'])
print("="*80)


🔍 Retrieved 5 items:
   - Vegetable Dosa (score: 0.570)
   - Dry Rub (score: 0.567)
   - Iced Dirty Chai Tea Latte Whole (score: 0.564)
   - Masala Dosa (score: 0.563)
   - Tostones (score: 0.562)
 RESPONSE:
For a dinner option that includes protein and supports digestion, I recommend the **Masala Dosa at Tandoor Indian Cuisine** (340 calories, 10g protein, 6g fiber). With a decent protein content and good fiber, it helps with digestion while offering a flavorful experience.

Another option is the **Vegetable Dosa at Tandoor Indian Cuisine** (180 calories, 5g protein, 3g fiber). It’s lighter in calories and still provides a good amount of fiber, which is beneficial for digestion.

If you're looking for something different, the **Tostones at Sazon** (280 calories, 2g protein, 2g fiber) can be a tasty side, though the protein content is lower. Pair it with the Masala Dosa for a balanced meal that enhances your protein intake and digestion!


## Test Multiple Queries

These match the rubric requirements!

In [655]:
#Test various nutrition goals
test_queries = [
    "I want to bulk up and gain muscle",
    "What's good for weight loss?",
    "I need something for pre-workout energy",
    "I'm doing keto, what can I eat?",
    "I need more fiber in my diet",
    "Low sodium options for heart health?",
    "What's a healthy breakfast at Duke?",
    "I'm vegetarian and need protein"
]

print(" TESTING RAG SYSTEM WITH VARIOUS QUERIES\n")
print("="*80)

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*80}")
    print(f"Query {i}/{len(test_queries)}: {query}")
    print("="*80)
    
    result = rag.ask(query, k=5, use_history=False, verbose=False)
    
    print(f"\nResponse:")
    print(result['response'])
    
    print(f"\nRetrieved items:")
    for r in result['retrieved_items'][:3]:
        it = r["item"] 
        print(
            f"   - {it['item_name']} at {it.get('restaurant', 'Unknown')} "
            f"(score: {r['score']:.3f})"
        )

print("\n" + "="*80)
print("All test queries complete!")

 TESTING RAG SYSTEM WITH VARIOUS QUERIES


Query 1/8: I want to bulk up and gain muscle

Response:
For bulking up and gaining muscle, it's important to choose meals with a moderate protein ratio and sufficient calories. Here are three great options from the Duke dining halls:

1. **Turkey and Provolone Wrap at Red Mango**: This wrap offers 37g of protein for 570 calories, which gives it a protein ratio of about 26%. While it's slightly higher in calories, it provides a good balance of protein and carbohydrates for energy, making it suitable for muscle gain.

2. **Beef Sirloin Tri Tip with Gravy at Duke Marine Lab**: With 11g of protein and only 100 calories, this option has a protein ratio of 44%, which is excellent when considering calorie density. Pairing this with a higher-carb side could enhance your overall intake for bulking.

3. **Chicken at The Pitchfork**: This grilled chicken option provides 13g of protein for just 120 calories, resulting in a protein ratio of about 43%. It's

## Multi-Turn Conversation Test

In [657]:
# Reset conversation
rag.reset_conversation()

print(" MULTI-TURN CONVERSATION TEST\n")
print("="*80)

# Turn 1
print("\n User: I want to lose weight")
result1 = rag.ask("I want to lose weight", k=5, use_history=True, verbose=False)
print(f"\n Assistant: {result1['response']}")

# Turn 2 (follows up on Turn 1)
print("\n" + "-"*80)
print("\n User: What about for breakfast specifically?")
result2 = rag.ask("What about for breakfast specifically?", k=5, use_history=True, verbose=False)
print(f"\n Assistant: {result2['response']}")

print("\n" + "-"*80)
print("\n User: I am not able to eat at Marketplace")
result25 = rag.ask("I am not able to eat at Marketplace", k=5, use_history=True, verbose=False)
print(f"\n Assistant: {result25['response']}")

# Turn 3 (continues conversation)
print("\n" + "-"*80)
print("\n User: Are any of those vegetarian?")
result3 = rag.ask("Are any of those vegetarian?", k=5, use_history=True, verbose=False)
print(f"\n Assistant: {result3['response']}")

print("\n" + "="*80)
print("\n Multi-turn conversation working!")
print(" Context maintained across 3 turns")
print("\n Conversation history length:", len(rag.conversation_history))

 MULTI-TURN CONVERSATION TEST


 User: I want to lose weight

 Assistant: For your weight loss goals, I recommend the **Country Ham at The Skillet** (28g protein, 220 cal, 51% protein). It's a lean source of protein that will help keep you full without adding too many calories. 

You might also consider the **Applewood Smoked Bacon at Zweli's Café at Duke Divinity** (10g protein, 130 cal, 77% protein) for a tasty, low-calorie option, although it's lower in protein. 

Unfortunately, the other options are higher in calories and lower in protein-to-calorie ratios, making them less ideal for your weight loss journey.

--------------------------------------------------------------------------------

 User: What about for breakfast specifically?

 Assistant: For a weight loss breakfast, I recommend the **Iced Latte with Skim Milk at Trinity Cafe** (8g protein, 80 cal, 40% protein). It's low in calories and provides a nice protein boost to start your day. 

You could also try the **Chicken Sa

## Few-Shot Learning Example

In [659]:
few_shot_system_prompt = """You are a helpful nutrition assistant for Duke University students.

Here are examples of good recommendations:

Example 1:
Query: "I'm cutting what should I eat?"
Response: "For low-calorie options, the Egg White Scramble at Marketplace is perfect 
(120 calories, 25g protein). You'll stay full while keeping calories low. The Garden 
Salad from [] is another light choice under 150 calories! If you want something more filling, [] is also a great option with [] calories while still having a healthy set of nutrients

Example 2:
Query: "Is the Southwest Cobb Salad from The Devils Krafthouse healthy for my goals? I want to become slimmer while maintaining muscle. Be detailed"
Response: "For your goal of getting slimmer while maintaining muscle, the Southwest Cobb Salad from The Devils Krafthouse is generally a strong option, 
especially if you tweak it a bit. It’s built around lean protein (chicken/egg), which helps with muscle maintenance, and the veggies give you volume and fiber to keep you full on fewer calories.

Where it can get heavy is usually the dressing, cheese, and bacon. To make it more ‘cut-friendly,’ I’d suggest: asking for the dressing on the side or a lighter vinaigrette, keeping the protein (chicken/egg) as-is, and, if you want to shave off extra calories, 
going lighter on cheese or bacon. With those adjustments, it becomes a high-protein, relatively calorie-controlled meal that fits really well with your goal of getting lean while keeping muscle"

Example 3:
Query: "I need a carb-heavy meal I'm running 8 miles after lunch."
Response: "For an 8-mile run, you want meals that are carb-dominant for glycogen, with moderate protein to stabilize energy.

Chicken Teriyaki Rice Bowl @ Marketplace

62g carbs · 28g protein · 520 calories
The rice gives you a fast yet steady release of carbs, while the chicken adds enough protein to keep your blood sugar stable. The teriyaki sauce provides quick energy, which is great pre-run. This is ideal if you need fuel that kicks in within 1–2 hours.

Pesto Pasta Bowl @ The Loop

74g carbs · 16g protein · 580 calories
Pasta is pure endurance fuel. Most calories come from carbs, which means this will effectively top off muscle glycogen before a long run. The pesto gives you a bit of fat for longer-lasting energy, so you don’t crash halfway through.

Sweet Potato & Black Bean Plate @ The Refectory

68g carbs · 14g protein · 490 calories
This option is very gut-friendly and has slow-digesting carbs. Sweet potatoes are one of the best pre-run foods because they give sustained energy without stomach discomfort. The fiber is moderate, not too high, so it won’t interfere with your run.

Tip: Eat 2–3 hours before your run to allow time for digestion, and sip water consistently afterward."


Example 4:
Query:
"I’m only eating at Pitchfork or JB’s today. I want something high protein but not too many calories."

Response:
"Grilled Chicken Breast @ JB’s

40g protein · 260 calories · ~62% protein
This is one of the leanest entrées on campus. Nearly all the calories come from protein, making it perfect for cutting or maintaining muscle without going over calories. The sides are usually steamed vegetables or a simple starch, so you can keep it clean and portion-controlled.

Turkey Pesto Flatbread (No Cheese) @ The Pitchfork

36g protein · 310 calories · strong protein density
Removing the cheese drops both the fat and total calories, making the turkey the star of the dish. Because the base is still a flatbread, you get enough carbs to feel satisfied without the meal becoming heavy. Great balance if you want high protein but still need some energy.

Blackened Tilapia Bowl @ JB’s

34g protein · 280 calories · very lean
Tilapia is naturally low-fat and high-protein, and the blackened seasoning adds flavor without extra calories. This bowl is extremely macro-efficient — light enough not to weigh you down, but substantial enough to help maintain muscle on a calorie deficit."


Example 5: 
Query: "I need something low-sodium for heart health, and please avoid anything with sauces or dressings. I can’t eat at Marketplace."

Response: "For low-sodium and heart-friendly meals that are not at Marketplace, here are smart options that naturally keep salt and sauces to a minimum:

1. Grilled Salmon Plate @ The Commons
Nutrition: 310 calories · 34g protein · 120mg sodium
Salmon is naturally low in sodium when served plain, and it’s rich in omega-3 fats, making it ideal for heart health.

2. Herb-Roasted Chicken Breast @ JB’s
Nutrition: 260 calories · 42g protein · 105mg sodium
This entrée is prepared without sauce and keeps salt levels low while delivering very lean protein.

3. Steamed Veggie & Quinoa Bowl @ Trinity Café
Nutrition: 280 calories · 10g protein · 95mg sodium
Quinoa and steamed vegetables keep sodium extremely low while offering complex carbs and fiber. Perfect for a gentle, heart-healthy meal.

Tip: Ask for “no seasoning blend” since many house blends contain added sodium."
"""

# Test few-shot prompting
print(" TESTING FEW-SHOT LEARNING\n")
print("="*80)

# Temporarily update system prompt
original_prompt = rag.system_prompt
rag.system_prompt = few_shot_system_prompt

test_query = "I need post-workout recovery food"
print(f"\n Query: {test_query}")

result = rag.ask(test_query, k=5, use_history=False, verbose=False)

print(f"\n Response (with few-shot examples):")
print(result['response'])

rag.system_prompt = original_prompt

print("\n" + "="*80)
print(" Few-shot learning demonstrated")
print(" This shows in-context learning")

 TESTING FEW-SHOT LEARNING


 Query: I need post-workout recovery food

 Response (with few-shot examples):
For post-workout recovery, you'll want a meal that includes both protein for muscle repair and carbohydrates to replenish glycogen stores. The Personal Cheese Pizza at Marketplace can be part of this recovery meal, but it might be beneficial to pair it with a higher protein option to optimize recovery. 

Here’s a suggestion:

**Personal Cheese Pizza @ Marketplace**
- **Calories:** 180
- **Protein:** 8g
- **Carbs:** 25g
- **Fat:** 5g
- **Fiber:** 2g

To make this meal more effective for recovery, consider adding a protein-rich side:

**Grilled Chicken Breast @ JB’s**
- **Calories:** 260
- **Protein:** 40g
- **Carbs:** 0g
- **Fat:** 5g

By combining the pizza with the grilled chicken breast, you’ll have a balanced post-workout meal:
- Total Calories: 440
- Total Protein: 48g
- Total Carbs: 25g
- Total Fat: 10g

This combination provides a good mix of carbs and protein, making it gr

## Interactive Demo


In [661]:
rag.reset_conversation()

# Interactive loop
print(" INTERACTIVE DEMO")
print("Type 'quit' to exit\n")
print("="*80)

while True:
    query = input("\n You: ")
    
    if query.lower() in ['quit', 'exit', 'q']:
        print("\n Thanks for using Duke Nutrition Assistant!")
        break
    
    if not query.strip():
        continue
    
    result = rag.ask(query, k=5, use_history=True, verbose=False)
    
    print(f"\n Assistant: {result['response']}")
    print("-"*80)

 INTERACTIVE DEMO
Type 'quit' to exit




 You:  quit



 Thanks for using Duke Nutrition Assistant!


## Save System for Later Use

In [675]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

conversation_log = {
    'timestamp': timestamp,
    'history': rag.conversation_history,
    'model': 'gpt-4o-mini',
    'system_prompt': rag.system_prompt
}

with open(f'models/conversation_log_{timestamp}.json', 'w') as f:
    json.dump(conversation_log, f, indent=2)

print(f" Conversation saved to: models/conversation_log_{timestamp}.json")

 Conversation saved to: models/conversation_log_20251208_162244.json


## Cost Tracking

In [678]:
# Estimate costs
# gpt-4o-mini: $0.150 per 1M input tokens, $0.600 per 1M output tokens

print(" COST ESTIMATE\n")
print("="*80)

# Very rough estimate
num_queries = 15  # Approximate from tests
avg_input_tokens = 500  # Context + query
avg_output_tokens = 150  # Response

total_input = num_queries * avg_input_tokens
total_output = num_queries * avg_output_tokens

input_cost = (total_input / 1_000_000) * 0.150
output_cost = (total_output / 1_000_000) * 0.600
total_cost = input_cost + output_cost

print(f"Estimated queries: {num_queries}")
print(f"Estimated total cost: ${total_cost:.4f}")
print("="*80)

 COST ESTIMATE

Estimated queries: 15
Estimated total cost: $0.0025


## Notebook 3 Contains:

-  Complete RAG system (retrieve + generate)
-  GPT-4 API integration
-  Multi-turn conversation with history
-  Prompt engineering with system prompts
-  Few-shot learning examples
-  Interactive demo

### Notebook 4 Next one should contain:
   - Quantitative metrics
   - Qualitative evaluation
   - Baseline comparisons
   - Ablation study

### Total Points So Far:
From Notebooks 1-3:
- Data collection: 10 pts
- RAG system: 10 pts
- Embeddings: 5 pts
- API calls: 5 pts
- Multi-turn: 7 pts
- Prompt engineering: 3 pts
- Few-shot: 5 pts
- Preprocessing: 3 pts